# multi-thread

[threading](https://docs.python.org/2/library/threading.html) 相比于[thread](https://www.tutorialspoint.com/python/python_multithreading.htm) 是一个功能更加多样的模块。主要可以利用该模块实现多任务的并行执行，以提高python的执行效率。

通常利用 [Producer/Consumer Threading Pattern/scenario](http://pages.cs.wisc.edu/~remzi/OSTEP/threads-cv.pdf)来规划多线程并行问题。Producer 通常用来生成任务并利用 queue 来传递任务给 Consumer， Consumer 一次从queue中认领一个任务并执行。非常类似MPI中的master-slave模式。

参考： http://chriskiehl.com/article/parallelism-in-one-line/

In [4]:
#标准的consumer/producer 模式
'''
Standard Producer/Consumer Threading Pattern
'''

import time 
import threading 
import Queue 

class Consumer(threading.Thread): 
  def __init__(self, queue): 
    threading.Thread.__init__(self)
    self._queue = queue 

  def run(self):
    while True: 
      # queue.get() blocks the current thread until 
      # an item is retrieved. 
      msg = self._queue.get() 
      # Checks if the current message is 
      # the "Poison Pill"
      if isinstance(msg, str) and msg == 'quit':
        # if so, exists the loop
        break
      # "Processes" (or in our case, prints) the queue item 
      print "I'm a thread %s, and I received %s!!" %(self.getName(), msg)
    # Always be friendly! 
    print 'Bye byes!'


def Producer():
  # Queue is used to share items between
  # the threads.
  queue = Queue.Queue()

  # Create an instance of the worker
  worker = Consumer(queue)
  # start calls the internal run() method to 
  # kick off the thread
  worker.start() 

  # variable to keep track of when we started
  start_time = time.time() 
  # While under 5 seconds.. 
  while time.time() - start_time < 5: 
    # "Produce" a piece of work and stick it in 
    # the queue for the Consumer to process
    queue.put('something at %s' % time.time())
    # Sleep a bit just to avoid an absurd number of messages
    time.sleep(0.5)

  # This the "poison pill" method of killing a thread. 
  queue.put('quit')
  # wait for the thread to close down
  worker.join()


if __name__ == '__main__':
  Producer()

I'm a thread Thread-7, and I received something at 1509502066.46!!
I'm a thread Thread-7, and I received something at 1509502066.96!!
I'm a thread Thread-7, and I received something at 1509502067.46!!
I'm a thread Thread-7, and I received something at 1509502067.96!!
I'm a thread Thread-7, and I received something at 1509502068.46!!
I'm a thread Thread-7, and I received something at 1509502068.96!!
I'm a thread Thread-7, and I received something at 1509502069.46!!
I'm a thread Thread-7, and I received something at 1509502069.96!!
I'm a thread Thread-7, and I received something at 1509502070.46!!
I'm a thread Thread-7, and I received something at 1509502070.96!!
Bye byes!


chriskiehl 认为需要一个更加简单的类可以做各种事情，同时需要能够管理队列（queue）

In [1]:
import time
import threading
import Queue

class Consumer(threading.Thread):
    def __init__(self, queue):
        threading.Thread.__init__(self)
        self._queue = queue

    def run(self):
        while True:
            content = self._queue.get()
            if isinstance(content, str) and content == 'quit':
                break
            # "Processes" (or in our case, prints) the queue item
            print "I'm a thread %s, and I received %s\n" %(self.getName(),
                                                           content)
        print 'Bye byes!\n'


def Producer():
    queue = Queue.Queue()
    worker_threads = build_worker_pool(queue, 4)
    start_time = time.time()

    words = ['this', 'is', 'a', 'hello', 'world', 'test', '!']
    for word in words:
        queue.put(word)

    for _ in worker_threads:
        queue.put('quit')
    for worker in worker_threads:
        worker.join()

    print 'Done! Time taken: {}'.format(time.time() - start_time)

def build_worker_pool(queue, size):
    workers = []
    for _ in range(size):
        worker = Consumer(queue)
        worker.start()
        workers.append(worker)
    return workers

if __name__ == '__main__':
    Producer()

I'm a thread Thread-5, and I received this

I'm a thread Thread-5, and I received hello
I'm a thread Thread-4, and I received is
I'm a thread Thread-7, and I received a
I'm a thread Thread-6, and I received world




Done! Time taken: 0.0322029590607I'm a thread Thread-5, and I received test
I'm a thread Thread-4, and I received !
Bye byes!
Bye byes!





Bye byes!
Bye byes!




但是上述方法还是有点复杂，最简单的方式是利用Map函数来并行执行上述过程

In [3]:
from multiprocessing.dummy import Pool as ThreadPool
import time

secds = [
    '1',
    '1',
    '1',
    '1',
    '1',
    '1',
    '1',
]

def sleep(url):
    time.sleep(int(url))

num = 8
# Make the Pool of workers
pool = ThreadPool(num)
# Open the urls in their own threads
# and return the results
start_time = time.time()
results = pool.map(sleep, secds)
print 'Done! Time taken with {} thread: {}'.format(num, time.time() -
                                                   start_time)

#close the pool and wait for the work to finish
pool.close()
pool.join()

Done! Time taken with 8 thread: 1.00190496445
